In [11]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import re
import copy
import numpy as np

In [2]:
core=pd.read_csv('ModelList.txt', sep="\t", index_col=0, header=None)
for i in core.index:
    core.rename(index={i:re.sub('\.xml','',i)}, inplace=True)
genera=core.index

In [17]:
def open_model (modelfile):
    model = cobra.io.read_sbml_model(modelfile)
    model.solver='glpk'
    return model

In [31]:
def get_essentiality (model, samples, pinEX, pinALL):
    
    #close all exchanges
    with_ex={}
    xch=[]
    complete_media=[]
    cm=[]
    met_name=[]
    for e in model.reactions:
        if re.search('EX_',e.id):
            complete_media.append(e)
            
            cpd=e.id
            cpd=re.sub('EX_','',cpd)
            met_name.append(model.metabolites.get_by_id(cpd).name)
            cpd=re.sub("_\w0",'',cpd)
            cm.append(cpd)
            
            with_ex[cpd]=e;
            e.lower_bound=0
            xch.append(e)
            
    
    #add_exchanges for every metabolite
    for m in model.metabolites:
        cpd=re.sub("_\w0",'',m.id)
        if cpd in with_ex:
            continue
        newex='EX_'+m.id
        reaction=Reaction(newex)
        reaction.name=m.id+' exchange'
        reaction.subsystem=''
        reaction.lower_bound=0
        reaction.upper_bound=0
        reaction.gene_reaction_rule='Unknown'
        model.add_reactions([reaction])
        reaction.add_metabolites({m:-1})
        xch.append(reaction)
    
    #allow export of metabolites with exchanges
    for e in with_ex:
        with_ex[e].upper_bound=1000
        
    #Ensure at least x biomass is produced
    model.reactions.biomass1.lower_bound=0.01
    

    count=0
    results=pd.DataFrame(index=list(with_ex.keys()), columns=['essential', 'counts','p_essential','name'], 
                         data=np.zeros((len(with_ex.keys()),4)))
    results.loc[:,'name']=met_name
    
    #test essentiality of each metabolite under a specified number of random media compositions
    
    while count < samples:
        
        #define random media (metabolites without exchanges)
        lower_bounds=(np.random.rand(len(xch))<=pinALL).astype(int)*-1000
        for e in range(len(xch)):
            xch[e].lower_bound=lower_bounds[e]    
        
        #define random media (metabolites with exchanges)
        lower_bounds=(np.random.rand(len(complete_media))<=pinEX).astype(int)*-1000
        present=[]
        for e in range(len(complete_media)):
            complete_media[e].lower_bound=lower_bounds[e]
            if lower_bounds[e]<0:
                present.append(cm[e])
        
        #nblock biomass uptake        
        model.reactions.EX_cpd11416_c0.lower_bound=0
        
        #test media viability
        try: #do parsimonious fba
            solution=cobra.flux_analysis.pfba(model)
        except:
            continue
        if solution.status=='infeasible':
            continue
        count+=1
        results.loc[present,'counts']+=1
        
        # evaluate metabolite essentiality on viable media
        for exchr in present:
            #close uptake
            with_ex[exchr].lower_bound=0
            try: #do parsimonious fba
                solution=cobra.flux_analysis.pfba(model)
            except:
                results.loc[exchr,'essential']+=1
                with_ex[exchr].lower_bound=-1000
                continue
            if solution.status=='infeasible':
                results.loc[exchr,'essential']+=1
            with_ex[exchr].lower_bound=-1000
        print(count)
    results.loc[:,'p_essential']=(results.essential/results.counts).values
    return results

In [32]:
#probability that metabolites with exchange reactions are available in random media
pinEX=0.9
#probability that metabolites without exchange reactions are available in random media
pinALL=0
#number of viable random media to test
nsamples=10

In [ ]:
#run for first model
query=genera[1]
modelfile=query+'.xml'
model=open_model(modelfile)
R=get_essentiality(model, nsamples, pinEX, pinALL)

In [36]:
# save output
outname=query+'.essentiality.'+str(nsamples)+'.'+str(pinEX)+'_'+str(pinALL)+'.txt'
R.to_csv(outname, sep ="\t")